In [5]:
from sklearn.datasets import load_iris
from sklearn.datasets import load_sample_images
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

# 1. PCA

In [6]:
iris = load_iris()

In [7]:
X = iris.data

In [8]:
X_centered = X - X.mean(axis=0)

In [9]:
U, s, Vt = np.linalg.svd(X_centered)
c1 = Vt.T[:, 0]
c2 = Vt.T[:, 1]

In [10]:
c1

array([ 0.36158968, -0.08226889,  0.85657211,  0.35884393])

In [11]:
c2

array([-0.65653988, -0.72971237,  0.1757674 ,  0.07470647])

In [12]:
W2 = Vt.T[:, :2]
X2D = X_centered.dot(W2)

In [13]:
pca = PCA(n_components=2)
X2D = pca.fit_transform(X)

In [14]:
pca.explained_variance_ratio_

array([ 0.92461621,  0.05301557])

In [15]:
# choose right dim
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
len(pca.explained_variance_ratio_)

4

In [16]:
# you can set n_components to be a float between 0.0 and 1.0, indicating the ratio of variance you wish to preserve
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

# 2. PCA for Compression

In [17]:
pca = PCA(n_components = 2)
X_reduced = pca.fit_transform(X)
X_recovered = pca.inverse_transform(X_reduced)

In [18]:
# For mnist data, header must be None

In [4]:
mnist = pd.read_csv("/Users/lgrcyanny/Codecookies/machine-learning-workspace/datasets/mnist/mnist_train.csv", header=None)

In [82]:
mnist = mnist[0:100]

In [19]:
X = mnist.iloc[:, 1:]
y = mnist.iloc[:, :1]

In [20]:
from sklearn.decomposition import IncrementalPCA
n_batches = 10
inc_pca = IncrementalPCA(n_components=154)
for X_batch in np.array_split(X, n_batches):
    inc_pca.partial_fit(X_batch)
X_reduced = inc_pca.transform(X)

# 3. Kernal PCA

In [ ]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(mnist)

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris

In [ ]:
clf = Pipeline([
        ("kpca", KernelPCA(n_components=2)),
        ("log_reg", LogisticRegression())
    ])

param_grid = [{
        "kpca__gamma": np.linspace(0.03, 0.05, 10),
        "kpca__kernel": ["rbf", "sigmoid"]
    }]

grid_search = GridSearchCV(clf, param_grid, cv=3)

In [ ]:
grid_search.fit(X, y)

In [ ]:
grid_search.best_params_

In [ ]:
# reconstruction
rbf_pca = KernelPCA(n_components = 2, kernel="rbf", gamma=0.0500,
                    fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(X, X_preimage)